In [3]:
!pip3 install git+https://github.com/googleapis/python-aiplatform.git

  Cloning https://github.com/googleapis/python-aiplatform.git to /private/var/folders/4x/m777f74j7bbdl_9hqrqnqrpw0000gp/T/pip-req-build-mp6gdnzi
  Running command git clone --filter=blob:none --quiet https://github.com/googleapis/python-aiplatform.git /private/var/folders/4x/m777f74j7bbdl_9hqrqnqrpw0000gp/T/pip-req-build-mp6gdnzi
  Resolved https://github.com/googleapis/python-aiplatform.git to commit 9c3ad838a08ab014f06680fb84ccb7e280960f71
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for google-cloud-aiplatform: filename=google_cloud_aiplatform-1.72.0-py2.py3-none-any.whl size=6205634 sha256=1fec43888f31e81be03117da8f323ba8c67ff049f5f6564fc133544303d56671
  Stored in directory: /private/var/folders/4x/m777f74j7bbdl_9hqrqnqrpw0000gp/T/pip-ephem-wheel-cache-ao_18jg1/wheels/d5/a0/d4/be5e42761b480b0e7bf566a22df701defb3885e8420a7cb81b
Successfully built google-cloud-aiplatform
  Attem

In [48]:
from google.cloud import aiplatform

import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/lmattingly/Downloads/rv-hermes-nonprod-3a8067e91511.json"

# Replace with your project ID, region, and bucket name
project_id = "rv-hermes-nonprod"
region = "us-central1"

# Update when you need to create a new version
UID = "3"

# Initialize Vertex AI
aiplatform.init(project=project_id, location=region)

# Create a data store
dense_embedding_index = aiplatform.MatchingEngineIndex.create_tree_ah_index(
    display_name=f"commerce-product-search-index-poc_{UID}",
    contents_delta_uri="gs://commerce-product-search-data/json_vector_search_embeddings_format_11_09_2024.json",
    dimensions=768,
    approximate_neighbors_count=10,
    description="Data store for commerce product search",
)

# Create a search engine
my_index_endpoint = aiplatform.MatchingEngineIndexEndpoint.create(
    display_name=f"commerce-product-index-endpoint-poc_{UID}",
    public_endpoint_enabled=True
)



Creating MatchingEngineIndex
Create MatchingEngineIndex backing LRO: projects/3048625469/locations/us-central1/indexes/7309372981547892736/operations/2312518865860427776
MatchingEngineIndex created. Resource name: projects/3048625469/locations/us-central1/indexes/7309372981547892736
To use this MatchingEngineIndex in another session:
index = aiplatform.MatchingEngineIndex('projects/3048625469/locations/us-central1/indexes/7309372981547892736')
Creating MatchingEngineIndexEndpoint
Create MatchingEngineIndexEndpoint backing LRO: projects/3048625469/locations/us-central1/indexEndpoints/3531785280044400640/operations/6105675652013228032
MatchingEngineIndexEndpoint created. Resource name: projects/3048625469/locations/us-central1/indexEndpoints/3531785280044400640
To use this MatchingEngineIndexEndpoint in another session:
index_endpoint = aiplatform.MatchingEngineIndexEndpoint('projects/3048625469/locations/us-central1/indexEndpoints/3531785280044400640')


In [60]:
# List deployed indexes to verify
endpoint = aiplatform.MatchingEngineIndexEndpoint(index_endpoint_name=my_index_endpoint.resource_name)
deployed_indexes = endpoint.list()

# Print deployed index information
for deployed_index in deployed_indexes:
    print(f"Deployed Index Resource Name: {deployed_index.resource_name}, Index Name: {deployed_index.name}, Index ID: {print(deployed_index.to_dict())}")

{'name': 'projects/3048625469/locations/us-central1/indexEndpoints/3531785280044400640', 'displayName': 'commerce-product-index-endpoint-poc_3', 'deployedIndexes': [{'id': 'commerce_product_search_index_poc_3', 'index': 'projects/3048625469/locations/us-central1/indexes/7309372981547892736', 'createTime': '2024-11-10T23:17:50.237072Z', 'indexSyncTime': '2024-11-11T00:09:02.851941Z', 'automaticResources': {'minReplicaCount': 2, 'maxReplicaCount': 2}, 'deploymentGroup': 'default'}], 'etag': 'AMEw9yMhja1BzsxwKJT5EqRYjlM5Tg2-2dWuGrNsMhvy_xdJ9rVgNTo3y--XFDIYbvJS', 'createTime': '2024-11-10T23:17:40.184403Z', 'updateTime': '2024-11-10T23:17:40.891664Z', 'publicEndpointDomainName': '2017349162.us-central1-3048625469.vdb.vertexai.goog', 'encryptionSpec': {}}
Deployed Index Resource Name: projects/3048625469/locations/us-central1/indexEndpoints/3531785280044400640, Index Name: 3531785280044400640, Index ID: None
{'name': 'projects/3048625469/locations/us-central1/indexEndpoints/1809721372528607

In [49]:
# deploy the Index to the Index Endpoint (this takes upto 30 minutes)
# HAS TO BE UNDERSCORES does not like dashes or other characters
DEPLOYED_TOKEN_INDEX_ID = f"commerce_product_search__deployed_index_{UID}"

my_index_endpoint.deploy_index(
    index=dense_embedding_index, deployed_index_id=DEPLOYED_TOKEN_INDEX_ID
)


Deploying index MatchingEngineIndexEndpoint index_endpoint: projects/3048625469/locations/us-central1/indexEndpoints/3531785280044400640
Deploy index MatchingEngineIndexEndpoint index_endpoint backing LRO: projects/3048625469/locations/us-central1/indexEndpoints/3531785280044400640/operations/8090637187776774144
MatchingEngineIndexEndpoint index_endpoint Deployed index. Resource name: projects/3048625469/locations/us-central1/indexEndpoints/3531785280044400640


resource name: projects/3048625469/locations/us-central1/indexEndpoints/3531785280044400640

In [57]:
# get text embedding model
from vertexai.preview.language_models import TextEmbeddingModel

#model = TextEmbeddingModel.from_pretrained("text-embedding-004")
model = TextEmbeddingModel.from_pretrained("textembedding-gecko")



# wrapper
def get_dense_embedding(text):
    return model.get_embeddings([text])[0].values


# test it
get_dense_embedding("Big Screen TV")

[-0.0035271639935672283,
 -0.030608998611569405,
 -0.015883613377809525,
 -0.005066364537924528,
 -0.005092427134513855,
 -0.011433430016040802,
 -0.003941800445318222,
 0.02379780076444149,
 0.028864746913313866,
 0.012904484756290913,
 0.06276968121528625,
 -0.029003828763961792,
 -0.02657853066921234,
 -0.022518599405884743,
 0.02597796358168125,
 -0.03710658848285675,
 0.02797863818705082,
 -0.016653668135404587,
 0.03238348290324211,
 -0.007115708664059639,
 0.030816277489066124,
 -0.0025427581276744604,
 -0.02116364985704422,
 -0.019615069031715393,
 -0.02302222140133381,
 0.009490461088716984,
 0.06452789157629013,
 -0.0748070627450943,
 -0.015430617146193981,
 0.04962438717484474,
 -0.06137407198548317,
 0.01605449989438057,
 -0.062194425612688065,
 -0.017297958955168724,
 0.016498545184731483,
 -0.0647277981042862,
 -0.017562082037329674,
 -0.0062120286747813225,
 0.006970772985368967,
 0.0461108461022377,
 0.007137123961001635,
 -0.040757060050964355,
 -0.05858569219708443,
 

In [64]:
from google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint import (
    HybridQuery,
)

# create HybridQuery
query_text = "Big Screen TV"
query_emb = get_dense_embedding(query_text)
query = HybridQuery(
    dense_embedding=query_emb,
)

In [78]:
QUICKSTART_TOKEN_INDEX_NAME = f"vs-quickstart-index-11090446"

QUICKSTART_DEPLOYED_TOKEN_INDEX_ID = f"vs_quickstart_deployed_11090446"

QUICKSTART_TOKEN_INDEX_ID = "612520335647965184"

QUICKSTART_ENDPOINT_ID = "4029995988822261760"


#quickstart_index_endpoint = aiplatform.MatchingEngineIndexEndpoint(
#    index_endpoint_name=f"projects/{project_id}/locations/{region}/index_endpoint/{QUICKSTART_TOKEN_INDEX_ID}"
#)

my_index_endpoint = aiplatform.MatchingEngineIndexEndpoint(
    index_endpoint_name=f"{QUICKSTART_ENDPOINT_ID}"
)

# build a query request
response = my_index_endpoint.find_neighbors(
    deployed_index_id=QUICKSTART_DEPLOYED_TOKEN_INDEX_ID,
    queries=[query],
    num_neighbors=5,
)

print(response)

# print results
#for idx, neighbor in enumerate(response[0]):
    #print(f"NeighborId neighbor_{idx}: {neighbor.id}")
    #title = df.title[int(neighbor.id)]
    #print(f"{title:<40}")

[[MatchNeighbor(id='28818', distance=0.07199648767709732, sparse_distance=None, feature_vector=[], crowding_tag='0', restricts=[], numeric_restricts=[], sparse_embedding_values=[], sparse_embedding_dimensions=[]), MatchNeighbor(id='1885', distance=0.06703688204288483, sparse_distance=None, feature_vector=[], crowding_tag='0', restricts=[], numeric_restricts=[], sparse_embedding_values=[], sparse_embedding_dimensions=[]), MatchNeighbor(id='18768', distance=0.06650601327419281, sparse_distance=None, feature_vector=[], crowding_tag='0', restricts=[], numeric_restricts=[], sparse_embedding_values=[], sparse_embedding_dimensions=[]), MatchNeighbor(id='25824', distance=0.06650601327419281, sparse_distance=None, feature_vector=[], crowding_tag='0', restricts=[], numeric_restricts=[], sparse_embedding_values=[], sparse_embedding_dimensions=[]), MatchNeighbor(id='26060', distance=0.06648629158735275, sparse_distance=None, feature_vector=[], crowding_tag='0', restricts=[], numeric_restricts=[], 

In [ ]:


# Send a search query
search_results = search_engine.search(query="whats the best laptop for gaming")

# Print the search results
for result in search_results:
    print(result)

In [40]:
#vs-quickstart-index-endpoint-11090446

Detected encoding: ascii (Confidence: 1.0)
The file is not encoded in UTF-8.


In [4]:
from google.cloud import aiplatform

import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/lmattingly/Downloads/rv-hermes-nonprod-3a8067e91511.json"
existing_index_id = "commerce-product-search-index-poc_3"  # Replace with your existing index ID
project_id = "rv-hermes-nonprod"
region = "us-central1"

# Initialize Vertex AI
aiplatform.init(project=project_id, location=region)

# Retrieve the existing index
embedding_index = aiplatform.MatchingEngineIndex(index_name="projects/rv-hermes-nonprod/locations/us-central1/indexes/7309372981547892736")


# Update the existing index with new contents

In [12]:
embedding_index.update_embeddings(contents_delta_uri="gs://commerce-product-search-data/vector_search_index")

Updating MatchingEngineIndex index: projects/3048625469/locations/us-central1/indexes/7309372981547892736
Update MatchingEngineIndex index backing LRO: projects/3048625469/locations/us-central1/indexes/7309372981547892736/operations/2964529261131595776


I0000 00:00:1731481977.447622 14624222 chttp2_transport.cc:1182] ipv4:142.250.72.10:443: Got goaway [1] err=UNAVAILABLE:GOAWAY received; Error code: 1; Debug Text: wrong_frame_sequence {created_time:"2024-11-13T00:12:57.445724-07:00", http2_error:1, grpc_status:14}


TimeoutError: Operation did not complete within the designated timeout of None seconds.